# Vícenásobná dědičnost a MRO

Python umožňuje, aby třída dědila z více rodičů najednou. Syntax je jednoduchá: `class Potomek(Rodic1, Rodic2):`.

S tím ale přichází otázka: **V jakém pořadí se volají metody rodičů?**

## 1. Jednoduchý příklad (Mixins)
Časté využití vícenásobné dědičnosti jsou tzv. **Mixins**. Třídy, které přidávají specifickou schopnost.

In [ ]:
class Zvire:
    def __init__(self, jmeno):
        self.jmeno = jmeno

class LetajiciMixin:
    def let(self):
        print(f"{self.jmeno} letí vysoko v oblacích!")

class PlavajiciMixin:
    def plav(self):
        print(f"{self.jmeno} plave pod vodou.")

# Kachna umí obojí + je to zvíře
class Kachna(Zvire, LetajiciMixin, PlavajiciMixin):
    pass

donald = Kachna("Donald")
donald.let()
donald.plav()

## 2. Diamantový problém a `super()`
Co se stane, když více rodičů definuje stejnou metodu (např. `__init__`)? Python používá linearizační algoritmus C3 k vytvoření **MRO (Method Resolution Order)**.

In [ ]:
class A:
    def pozdrav(self):
        print("A: Ahoj")
        # super() zde už v jednoduchém dědění nemá kam jít, 
        # ale v diamantu může předat řízení vedlejší větvi!

class B(A):
    def pozdrav(self):
        print("B: Začátek")
        super().pozdrav()
        print("B: Konec")

class C(A):
    def pozdrav(self):
        print("C: Začátek")
        super().pozdrav()
        print("C: Konec")

class D(B, C):
    def pozdrav(self):
        print("D: Začátek")
        super().pozdrav()
        print("D: Konec")

d = D()
print("--- Volání d.pozdrav() ---")
d.pozdrav()

### Analýza výstupu
Všimněte si, že ačkoliv `B` dědí z `A`, příkaz `super().pozdrav()` uvnitř `B` nezavolal `A`, ale `C`! 

Je to proto, že v kontextu třídy `D` vypadá fronta volání (MRO) takto:

In [ ]:
print(D.mro())

**Důležité pravidlo:** Aby MRO fungovalo správně a každá metoda se zavolala právě jednou, **všechny** metody v řetězci musí volat `super()`.